In [3]:
#Cloning Git repository

repo_url = "https://github.com/PhonePe/pulse.git"
destination = r"C:\Python\Pulse\Data"

from git import Repo

Repo.clone_from(repo_url,destination)



GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/PhonePe/pulse.git C:\Python\Pulse\Data
  stderr: 'fatal: destination path 'C:\Python\Pulse\Data' already exists and is not an empty directory.
'

In [4]:
#Creating a reuseable function to iterate through the directory
import os

def extract_from_input():
    path = input("Enter your path: ")
    quarter_paths = []

    for state in os.listdir(path):
        state_path = os.path.join(path, state)

        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)

            for quarter in os.listdir(year_path):
                quarter_paths.append(os.path.join(year_path, quarter))

    return quarter_paths


In [6]:
#extracting data for aggregate transaction by deploying reuseable function
import json
import pandas as pd

collections = {"State": [],"Year": [],"Quarter": [],"Transaction_type": [],"Transaction_count": [],
    "Transaction_amount": []}

quarter_paths = extract_from_input()

for file_path in quarter_paths:
    parts = file_path.split(os.sep)

    state = parts[-3]
    year = parts[-2]
    quarter = parts[-1].replace(".json", "")

    with open(file_path, "r") as f:
        d = json.load(f)

    for i in d["data"]["transactionData"]:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Transaction_type"].append(i["name"])
        collections["Transaction_count"].append(i["paymentInstruments"][0]["count"])
        collections["Transaction_amount"].append(i["paymentInstruments"][0]["amount"])

agg_transaction = pd.DataFrame(collections)
agg_transaction.to_csv("agg_transaction.csv", index = False)

FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

In [8]:
#extracting data for aggregate insurance by deploying reuseable function

collections = {"State": [], "Year": [], "Quarter": [], "name": [],
               "Count": [], "Amount": []}

quarter_paths = extract_from_input()

for file_path in quarter_paths:
    parts = file_path.split(os.sep)
    #print(parts)

    state = parts[-3]
    year = parts[-2]
    quarter = parts[-1].replace(".json", "")

    with open(file_path, "r") as f:
        d = json.load(f)
        #print(d)

    for i in d["data"]["transactionData"]:
                    collections["State"].append(state)   
                    collections["Year"].append(year)
                    collections["Quarter"].append(quarter.replace(".json",""))
                    collections["name"].append(i["name"])
                    collections["Count"].append(i["paymentInstruments"][0]["count"])
                    collections["Amount"].append(i["paymentInstruments"][0]["amount"])


agg_insurance = pd.DataFrame(collections)
agg_insurance.to_csv("agg_insurance.csv", index = False)

In [9]:
#Extracting JSON data from Aggregated user list using path lib and for loop
from pathlib import Path
import json
import pandas as pd

path = r'C:\Python\Pulse\Data\data\aggregated\user\country\india\state'

base_path = Path(path)

collections = {"State": [],"Year": [], "Quarter": [], "registeredUsers": [],
               "brand": [], "count": [], "percentage":[]}

for file_path in base_path.rglob("*.json"):
    state = file_path.parts[-3]
    year = file_path.parts[-2]
    quarter = file_path.stem

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    # Total users - to avoid error when null
    total_registered = data["data"]["aggregated"].get("registeredUsers", 0)
    #print(total_registered)

    # retriving devices list - to avoid error when null
    devices = data["data"].get("usersByDevice") or []
    #print(devices)

    for device in devices:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["registeredUsers"].append(total_registered)
        collections["brand"].append(device.get("brand", "Unknown"))
        collections["count"].append(device.get("count", 0))
        collections["percentage"].append(device.get("percentage", 0.0))



agg_users = pd.DataFrame(collections)
agg_users.to_csv("agg_users.csv", index = False)

In [12]:
##extracting data for MAP INSURANCE by deploying reuseable function
import json

collections = {"State": [],"Year": [], "Quarter": [], "Latitude": [],
               "Longitude": [], "Metric": [], "Districts":[]}

quarter_paths = extract_from_input()
#print(quarter_paths)
for file_path in quarter_paths:
    #print(file_path)
    with open(file_path, "r") as f:
        d = json.load(f)
        #print(d)
    
    parts = file_path.split(os.sep)
    state = parts[-3]
    year = parts [-2]
    quarter = parts[-1]

    for item in d["data"]["data"]["data"]:
        #print(item)
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter.replace(".json", ""))
        collections["Latitude"].append(item[0])
        collections["Longitude"].append(item[1])
        collections["Metric"].append(item[2])
        collections["Districts"].append(item[3])

map_insurance = pd.DataFrame(collections)
map_insurance.to_csv("map_insurance.csv", index = False)

    

In [11]:
#extracting data for MAP Transaction by deploying reuseable function

collections = {"State": [],"Year": [], "Quarter": [], "Districts": [], "Type": [],
               "Count": [], "Amount": []}

quarter_paths = extract_from_input()
#print(quarter_paths)

for file_path in quarter_paths:
    #print(file_path)
    parts = file_path.split(os.sep)
    #print(parts)
    state = parts[-3]
    year = parts[-2]
    quarter = parts[-1].replace(".json","")

    
    file_path = Path(file_path)

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)
        
        for item in data["data"]["hoverDataList"]:
            #print(item)
            collections["State"].append(state)
            collections["Year"].append(year)
            collections["Quarter"].append(quarter)
            collections["Districts"].append(item["name"])
            collections["Type"].append(item["metric"][0]["type"])
            collections["Count"].append(item["metric"][0]["count"])
            collections["Amount"].append(item["metric"][0]["amount"])

map_transaction = pd.DataFrame(collections)
map_transaction.to_csv("map_transaction.csv", index = False)

In [7]:
#extracting data for MAP USER by deploying reuseable function
import json

collections = {"State": [],"Year": [], "Quarter": [], "Districts": [], "RegisteredUsers": [],
               "appOpens": []}

quarter_paths = extract_from_input()

for file_path in quarter_paths:
    part = file_path.split(os.sep)
    #print(part)
    state = part[-3]
    year = part[-2]
    quarter = part[-1].replace(".json", "")
    
    
    with open(file_path, "r") as f:
        data = json.load(f)
    
    hover_data = data["data"].get("hoverData", {}) 
    #print(hover_data)   

    for district, values in hover_data.items():
         collections["State"].append(state)
         collections["Year"].append(year)
         collections["Quarter"].append(quarter)
         collections["Districts"].append(district)
         collections["RegisteredUsers"].append(values.get("registeredUsers", 0))
         collections["appOpens"].append(values.get("appOpens", 0))

map_users = pd.DataFrame(collections)
map_users.to_csv("map_users.csv", index = False)


In [13]:
#extracting data for TOP INSURANCE by deploying reuseable function

from pathlib import Path

path = r'C:\Python\Pulse\Data\data\top\insurance\country\india\state'
base_path= Path(path) #converting string into path file
base_path

collections = {
    "State": [],
    "Year": [],
    "Quarter": [],
    "Level": [],          # district / pincode
    "EntityName": [],     # district name or pincode
    "Type": [],
    "Count": [],
    "Amount": []

}
for file_path in base_path.rglob("*.json"):
    state = file_path.parts[-3]
    year = file_path.parts[-2]
    quarter = file_path.stem

    with file_path.open("r", encoding = "utf-8") as f:
        d = json.load(f)
        
    top_data = d["data"]

    # ---------- DISTRICT LEVEL ----------
    districts = top_data.get("districts") or []

    for d in districts:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("District")
        collections["EntityName"].append(d["entityName"])
        collections["Type"].append(d["metric"]["type"])
        collections["Count"].append(d["metric"]["count"])
        collections["Amount"].append(d["metric"]["amount"])

    # ---------- PINCODE LEVEL ----------
    pincodes = top_data.get("pincodes") or []

    for p in pincodes:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("Pincode")
        collections["EntityName"].append(p["entityName"])
        collections["Type"].append(p["metric"]["type"])
        collections["Count"].append(p["metric"]["count"])
        collections["Amount"].append(p["metric"]["amount"])


# Create DataFrame
top_insurance = pd.DataFrame(collections)
top_insurance.to_csv("top_insurance.csv", index = False)


In [14]:

#Extracting JSON data from TOP TRANSACTION user list using path lib and for loop


from pathlib import Path
import json
import pandas as pd

# ---------- COLLECTION ----------
collections = {
    "State": [],
    "Year": [],
    "Quarter": [],
    "Level": [],          # District / Pincode
    "EntityName": [],
    "Type": [],
    "Count": [],
    "Amount": []
}

# ---------- PATH ----------
path = r"C:\Python\Pulse\Data\data\top\transaction\country\india\state"
base_path = Path(path) #converting string into path file

# ---------- LOOP ----------
for file_path in base_path.rglob("*.json"):

    state = file_path.parts[-3]
    year = file_path.parts[-2]
    quarter = file_path.stem

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    top_data = data["data"]

    # ---------- DISTRICTS ----------
    districts = top_data.get("districts") or []

    for d in districts:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("District")
        collections["EntityName"].append(d["entityName"])
        collections["Type"].append(d["metric"]["type"])
        collections["Count"].append(d["metric"]["count"])
        collections["Amount"].append(d["metric"]["amount"])

    # ---------- PINCODES ----------
    pincodes = top_data.get("pincodes") or []

    for p in pincodes:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("Pincode")
        collections["EntityName"].append(p["entityName"])
        collections["Type"].append(p["metric"]["type"])
        collections["Count"].append(p["metric"]["count"])
        collections["Amount"].append(p["metric"]["amount"])


# ---------- DATAFRAME ----------
top_transaction = pd.DataFrame(collections)
top_transaction.to_csv("top_transaction.csv", index = False)


In [15]:
#Extracting JSON data from TOP USERS list using path lib and for loop


from pathlib import Path
import json
import pandas as pd

# ---------- COLLECTION ----------
collections = {
    "State": [],
    "Year": [],
    "Quarter": [],
    "Level": [],          # District / Pincode
    "Name": [],
    "RegisteredUsers": []
}

# ---------- PATH ----------
path = r"C:\Python\Pulse\Data\data\top\user\country\india\state"
base_path = Path(path) #converting string into path file

# ---------- LOOP ----------
for file_path in base_path.rglob("*.json"):

    state = file_path.parts[-3]
    year = file_path.parts[-2]
    quarter = file_path.stem

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    top_data = data["data"]

    # ---------- DISTRICT LEVEL ----------
    districts = top_data.get("districts") or []

    for d in districts:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("District")
        collections["Name"].append(d["name"])
        collections["RegisteredUsers"].append(d["registeredUsers"])

    # ---------- PINCODE LEVEL ----------
    pincodes = top_data.get("pincodes") or []

    for p in pincodes:
        collections["State"].append(state)
        collections["Year"].append(year)
        collections["Quarter"].append(quarter)
        collections["Level"].append("Pincode")
        collections["Name"].append(p["name"])
        collections["RegisteredUsers"].append(p["registeredUsers"])

# ---------- DATAFRAME ----------
top_users = pd.DataFrame(collections)
top_users.to_csv("top_users.csv", index = False)
